In [6]:
class TSP_NearestNeighbor:
    """
    단순하고 빠르지만 최적해는 보장하지 않는 탐욕(Greedy) 알고리즘
    비교군(Baseline)으로 사용합니다.
    """
    def __init__(self, dist_matrix):
        self.dist = dist_matrix
        if isinstance(dist_matrix, pd.DataFrame):
            self.n = len(dist_matrix)
        else:
            self.n = dist_matrix.shape[0]

    def run(self):
        if self.n <= 1: return [0, 0], 0
            
        unvisited = set(range(1, self.n))
        current = 0
        path = [0]
        total_cost = 0
        
        get_dist = lambda i, j: self.dist.iloc[i, j] if isinstance(self.dist, pd.DataFrame) else self.dist[i][j]

        while unvisited:
            next_node = min(unvisited, key=lambda x: get_dist(current, x))
            total_cost += get_dist(current, next_node)
            path.append(next_node)
            unvisited.remove(next_node)
            current = next_node
        
        total_cost += get_dist(current, 0)
        path.append(0)
        return path, total_cost

In [7]:
import numpy as np

NEG = -1e12  # numerical -inf


class TSPHypercubeBCJR_SOVA:
    """
    PDF 원래 식 기반으로 다시 정리한 BCJR + SOVA 버전.

    - 단일 trellis ψ[t, mask, last]만 사용 (with-β)
    - α_t(a)는 항상 ψ_{t-1}에서 직접 계산:
        α_t(a) = max_{m_{t-1}, last ∈ m_{t-1}}
                    [ ψ_{t-1}(m_{t-1}, last) + s(last, a) ]
      (여기에는 β_t는 안 들어가고, 과거 시점 ≤ t-1의 β 정보만 포함)
    - trellis → X_{it} 메시지:
        ζ_{it}(a) = α_t(a) + β_t(a) - λ_{it}(a)
      (∑_{i'≠i} λ_{i't}(a) = β_t(a) - λ_{it}(a) 사용한 형태)
    - ψ는 with-β forward trellis:
        ψ_t(m_t, a_t) = max_{m_{t-1}, a_{t-1}}
            [ ψ_{t-1}(m_{t-1}, a_{t-1}) + s(a_{t-1}, a_t) + β_t(a_t) ]
      (코드에서는 t=1..T, β_t(a_t)를 beta[t-1, a_t]에 매핑)
    - BCJR backward bwd[t, mask, last]와 함께
      Γ_t(m_t,last) = ψ_t(m_t,last) + bwd_t(m_t,last)로 SOVA LLR 계산.
    """

    def __init__(self, D, start_city=None,
                 damping=0.3, iters=200, verbose=False,
                 tiny_tiebreak=False, seed=0,
                 patience_no_cost_change=10, cost_tol=1e-12,
                 kappa_bcjr=1.0,
                 damp_L=0.5, damp_beta=0.5, damp_zeta=0.5):

        D = np.array(D, dtype=float)
        assert D.shape[0] == D.shape[1], "D must be square"
        C = D.shape[0]

        if start_city is None:
            start_city = 0
        start_city = int(start_city)
        assert 0 <= start_city < C

        # permute: start -> last (internal depot)
        perm = np.arange(C)
        if start_city != C - 1:
            perm[start_city], perm[C - 1] = perm[C - 1], perm[start_city]
        inv_perm = np.empty(C, dtype=int)
        inv_perm[perm] = np.arange(C)

        self.orig_D = D
        self.D = D[perm][:, perm]
        self.perm = perm
        self.inv_perm = inv_perm

        self.C = C
        self.N = C - 1
        self.depot = C - 1

        self.verbose = verbose
        self.damp = float(damping)
        self.iters = int(iters)
        self.tiny_tiebreak = bool(tiny_tiebreak)
        self.rng = np.random.default_rng(seed)
        self.patience_no_cost_change = int(patience_no_cost_change)
        self.cost_tol = float(cost_tol)

        # BCJR / SOVA 관련 파라미터
        self.kappa_bcjr = float(kappa_bcjr)
        self.damp_L = float(damp_L)
        self.damp_beta = float(damp_beta)
        self.damp_zeta = float(damp_zeta)

        # similarity (bigger is better)
        mx = np.max(self.D)
        self.s = mx - self.D

        # trellis 크기
        self.T = self.N
        self.M = 1 << self.N

        # 단일 forward trellis ψ (with-β)
        self.psi = np.full((self.T + 1, self.M, self.N), NEG)
        self.backptr = np.full((self.T + 1, self.M, self.N, 2), -1, dtype=int)

        # backward trellis (with-β, closure 포함)
        self.bwd_wb = np.full((self.T + 1, self.M, self.N), NEG)

        # α_t(a): ψ_{t-1}로부터 계산
        self.alpha = np.full((self.T, self.N), NEG)

        # simplified messages
        self.gamma_tilde = np.zeros((self.N, self.T))
        self.omega_tilde = np.zeros((self.N, self.T))
        self.phi_tilde   = np.zeros((self.N, self.T))
        self.eta_tilde   = np.zeros((self.N, self.T))
        self.rho_tilde   = np.zeros((self.N, self.T))
        self.delta_tilde = np.zeros((self.N, self.T))

        # λ_t(i,a), ζ_t(i,a), β_t(a)
        self.lambda_ = np.zeros((self.T, self.N, self.N))
        self.zeta    = np.zeros((self.T, self.N, self.N))
        self.beta    = np.zeros((self.T, self.N))

        # damping 캐시
        self._L_prev    = [np.zeros((self.N, self.N)) for _ in range(self.T)]
        self._beta_prev = [np.zeros(self.N)            for _ in range(self.T)]
        self._zeta_prev = [np.zeros((self.N, self.N)) for _ in range(self.T)]

    # ===================== Public =====================
    def run(self):
        stable = 0
        last_cost = None
        best_route, best_cost = None, None

        for it in range(self.iters):
            # (1) forward trellis & α
            self._trellis_forward_and_alpha()

            # (2) backward trellis (primal with β + closure)
            self._trellis_backward_bcjr()

            # (3) BCJR-style SOVA LLR 계산 (T x N)
            llr_t = self._compute_sova_llr_from_bcjr()

            # (4) 메시지 업데이트
            self._update_phi_eta_rho()
            self._update_lambda_beta_zeta_delta(
                kappa=self.kappa_bcjr,
                llr_t=llr_t,
                damp_L=self.damp_L,
                damp_beta=self.damp_beta,
                damp_zeta=self.damp_zeta,
            )
            self._update_gamma_omega()

            if self.tiny_tiebreak:
                self.gamma_tilde += 1e-12 * self.rng.standard_normal(self.gamma_tilde.shape)

            # (5) route & cost
            route = self.estimate_route()
            cost = self._route_cost(route)

            if self.verbose:
                print(f"[{it+1:03d}] cost={cost:.12f} route={route}")

            # best primal 추적
            if best_cost is None or cost < best_cost:
                best_cost, best_route = cost, route

            # plateau early stop
            if last_cost is not None and abs(cost - last_cost) <= self.cost_tol:
                stable += 1
            else:
                stable = 0
            last_cost = cost

            if stable >= self.patience_no_cost_change:
                return best_route, best_cost

        return best_route, best_cost

    # ===================== Trellis (single ψ) & α =====================
    def _trellis_forward_and_alpha(self):
        """
        ψ_t(m_t, last) : with-β forward metric
        α_t(a)         : ψ_{t-1}로부터 계산 (현재 시점 β_t는 포함되지 않음)
        """
        self.psi.fill(NEG)
        self.backptr.fill(-1)
        self.alpha.fill(NEG)

        # t = 1
        # α_1(a) = s(depot, a)
        # ψ_1(m={a}, a) = s(depot, a) + β_1(a)
        for a in range(self.N):
            m = 1 << a
            gain = self.s[self.depot, a]
            self.alpha[0, a] = gain                    # α_1(a)
            self.psi[1, m, a] = gain + self.beta[0, a]  # ψ_1
            self.backptr[1, m, a] = (0, -1)             # 센티넬

        # t = 2..T
        full_mask = (1 << self.N) - 1
        for t in range(2, self.T + 1):
            # 1) α_t(a) 계산: ψ_{t-1}에서 오는 factor→A_t 메시지
            #    α_t(a) = max_{m_{t-1},last∈m_{t-1}, a∉m_{t-1}}
            #                 [ ψ_{t-1}(m_{t-1},last) + s(last,a) ]
            for a in range(self.N):
                best_alpha = NEG
                for mask in range(self.M):
                    if mask == 0 or mask.bit_count() != (t - 1):
                        continue
                    if mask & (1 << a):
                        continue  # 아직 방문 안 한 도시만
                    m = mask
                    while m:
                        last = (m & -m).bit_length() - 1
                        m ^= (1 << last)
                        cand = self.psi[t - 1, mask, last] + self.s[last, a]
                        if cand > best_alpha:
                            best_alpha = cand
                self.alpha[t - 1, a] = best_alpha

            # 2) ψ_t 전이 (with-β)
            for mask in range(self.M):
                if mask == 0 or mask.bit_count() != (t - 1):
                    continue
                for a in range(self.N):
                    if mask & (1 << a):
                        continue
                    new_mask = mask | (1 << a)
                    best = NEG
                    best_last = -1

                    m = mask
                    while m:
                        last = (m & -m).bit_length() - 1
                        m ^= (1 << last)
                        if last == a:
                            continue  # self-loop 금지

                        # ψ_{t-1} + s(last,a) + β_t(a)
                        cand = self.psi[t - 1, mask, last] + self.s[last, a] + self.beta[t - 1, a]
                        if cand > best:
                            best = cand
                            best_last = last

                    if best > self.psi[t, new_mask, a]:
                        self.psi[t, new_mask, a] = best
                        self.backptr[t, new_mask, a] = (mask, best_last)

    # ===================== Backward trellis (BCJR용) =====================
    def _trellis_backward_bcjr(self):
        """
        bwd_wb[t, mask, last]:
          - t 시점에 (mask,last) 상태에서 시작해서
          - 나머지 도시 방문 + depot으로 귀환까지의 최대 future metric.
        전이:
          - t < T:
              bwd[t,mask,last] = max_{a not in mask} { s[last,a] + β_{t+1}(a) + bwd[t+1, mask|{a}, a] }
            (코드에선 β_{t+1}(a)를 beta[t, a]로 저장)
          - t = T:
              full_mask 에 대해서만 closure: s[last, depot]
        """
        self.bwd_wb.fill(NEG)
        full_mask = (1 << self.N) - 1

        # t = T: full mask에서 depot으로 가는 closure
        t = self.T
        for last in range(self.N):
            mask = full_mask
            self.bwd_wb[t, mask, last] = self.s[last, self.depot]

        # t = T-1..1 역순
        for t in range(self.T - 1, 0, -1):
            for mask in range(self.M):
                if mask == 0 or mask.bit_count() != t:
                    continue
                for last in range(self.N):
                    if not (mask & (1 << last)):
                        continue

                    best = NEG
                    avail = (~mask) & full_mask
                    m = avail
                    while m:
                        a = (m & -m).bit_length() - 1
                        m ^= (1 << a)
                        new_mask = mask | (1 << a)
                        cand = self.s[last, a] + self.beta[t, a] + self.bwd_wb[t + 1, new_mask, a]
                        if cand > best:
                            best = cand
                    self.bwd_wb[t, mask, last] = best

    # ===================== SOVA-style LLR from BCJR =====================
    def _compute_sova_llr_from_bcjr(self):
        """
        llr[t, i] ≈
          max_{mask, last=i, |mask|=t+1} [ ψ[t+1, mask, i] + bwd[t+1, mask, i] ]
          - max_{mask, last≠i, |mask|=t+1} [ ψ[t+1, mask, last] + bwd[t+1, mask, last] ]
        여기서 내부 시간 인덱스(t+1)는 1..T 와 매칭, 외부 t는 0..T-1.
        """
        T, N, M = self.T, self.N, self.M
        llr = np.zeros((T, N), dtype=float)

        for t in range(1, T + 1):  # 내부 시간: 1..T
            for i in range(N):
                best_with = NEG
                best_without = NEG

                for mask in range(M):
                    if mask == 0 or mask.bit_count() != t:
                        continue

                    # last = i 인 state
                    val_i = self.psi[t, mask, i] + self.bwd_wb[t, mask, i]
                    if val_i > best_with:
                        best_with = val_i

                    # last ≠ i 인 state들 중 최고값
                    m2 = mask
                    while m2:
                        last = (m2 & -m2).bit_length() - 1
                        m2 ^= (1 << last)
                        if last == i:
                            continue
                        val = self.psi[t, mask, last] + self.bwd_wb[t, mask, last]
                        if val > best_without:
                            best_without = val

                if best_with <= NEG / 2 and best_without <= NEG / 2:
                    llr[t - 1, i] = 0.0
                else:
                    llr[t - 1, i] = best_with - best_without

        return llr

    # ===================== Messages =====================
    def _update_phi_eta_rho(self):
        # φ̃_it = -max_{i'≠i} γ̃_i't
        for t in range(self.T):
            col = self.gamma_tilde[:, t]
            for i in range(self.N):
                self.phi_tilde[i, t] = -np.max(np.delete(col, i)) if self.N > 1 else 0.0
        # η̃_it = -max_{t'≠t} ω̃_it'
        for i in range(self.N):
            row = self.omega_tilde[i, :]
            for t in range(self.T):
                self.eta_tilde[i, t] = -np.max(np.delete(row, t)) if self.T > 1 else 0.0
        # ρ̃_it
        self.rho_tilde = self.eta_tilde + self.phi_tilde

    def _update_lambda_beta_zeta_delta(self, kappa=0.0, llr_t=None,
                                       damp_L=0.5, damp_beta=0.5, damp_zeta=0.5):
        T, N = self.T, self.N

        for t in range(T):
            # (1) rhõ -> (rho0, rho1) 복원
            r = self.rho_tilde[:, t]  # shape (N,)
            rho0 = -r / N             # off-diagonal
            rho1 = rho0 + r           # diagonal

            # (2) L_new(i,a): a==i→rho1, else→rho0
            L_new = np.empty((N, N), float)
            for i in range(N):
                L_new[i, :] = rho0[i]
                L_new[i, i] = rho1[i]

            # (3) λ 이중 센터링
            L_new -= L_new.mean(axis=1, keepdims=True)
            L_new -= L_new.mean(axis=0, keepdims=True)

            # (4) λ damping
            L_prev = self._L_prev[t]
            L = damp_L * L_new + (1 - damp_L) * L_prev
            self.lambda_[t] = L

            # (5) β_t(a) = sum_i λ_{it}(a)
            beta_new = L.sum(axis=0)
            beta_new -= beta_new.mean()

            # (6) β damping
            beta_prev = self._beta_prev[t]
            beta_t = damp_beta * beta_new + (1 - damp_beta) * beta_prev
            self.beta[t, :] = beta_t

            # (7) ζ_it(a) = α_t(a) + β_t(a) - λ_it(a)
            a_t = self.alpha[t, :]
            Z_new = a_t[np.newaxis, :] + beta_t[np.newaxis, :] - L

            # (8) SOVA LLR 주입 (대각 성분)
            if kappa and llr_t is not None:
                for i in range(N):
                    Z_new[i, i] += kappa * llr_t[t, i]

            # (9) ζ damping
            Z_prev = self._zeta_prev[t]
            Z = damp_zeta * Z_new + (1 - damp_zeta) * Z_prev
            self.zeta[t] = Z

            # (10) δ̃_it = ζ_it(i) - max_{a≠i} ζ_it(a)
            for i in range(N):
                zi = Z[i, :]
                self.delta_tilde[i, t] = 0.0 if N == 1 else (zi[i] - np.max(np.delete(zi, i)))

        # 캐시 갱신
        self._L_prev    = [self.lambda_[t].copy() for t in range(T)]
        self._beta_prev = [self.beta[t].copy()    for t in range(T)]
        self._zeta_prev = [self.zeta[t].copy()    for t in range(T)]

    def _update_gamma_omega(self):
        gamma_new = self.eta_tilde + self.delta_tilde
        omega_new = self.phi_tilde + self.delta_tilde
        self.gamma_tilde = self.damp * gamma_new + (1 - self.damp) * self.gamma_tilde
        self.omega_tilde = self.damp * omega_new + (1 - self.damp) * self.omega_tilde

    # ===================== Decode =====================
    def estimate_route(self):
        full_mask = (1 << self.N) - 1
        best_val = NEG
        best_last = -1

        for last in range(self.N):
            base = self.psi[self.T, full_mask, last]
            if base <= NEG / 2:
                continue
            val = base + self.s[last, self.depot]  # closure
            if val > best_val:
                best_val = val
                best_last = last

        # backtrack
        if best_last < 0:
            # fallback: α 기반 greedy
            route_internal = [self.depot]
            used = set()
            for t in range(self.T):
                sc = self.alpha[t].copy()
                for u in used:
                    sc[u] = NEG
                if self.tiny_tiebreak:
                    sc += 1e-15 * np.arange(self.N)
                a = int(np.argmax(sc))
                used.add(a)
                route_internal.append(a)
            route_internal.append(self.depot)
        else:
            route_inner = []
            mask = full_mask
            last = best_last
            t = self.T
            while t > 0 and 0 <= last < self.N:
                route_inner.append(last)
                prev_mask, prev_last = self.backptr[t, mask, last]
                mask, last = prev_mask, prev_last
                t -= 1
            route_inner.reverse()
            route_internal = [self.depot] + route_inner + [self.depot]

        return [int(self.inv_perm[c]) for c in route_internal]

    def _route_cost(self, route):
        return float(sum(self.orig_D[route[k], route[k + 1]] for k in range(len(route) - 1)))


In [ ]:
import numpy as np
from collections import defaultdict

class TSPSolverSOVA:
    def __init__(self, dist_matrix, bp_iterations=20, damping=0.7, verbose=True):
        self.dist_matrix = np.array(dist_matrix)
        self.num_nodes = self.dist_matrix.shape[0]
        self.N = self.num_nodes - 1
        self.depot = self.N
        self.bp_iterations = bp_iterations
        self.damping = damping
        self.verbose = verbose
        self.INF = 1e12
        self.FULL_MASK = (1 << self.N) - 1
        
        max_dist = np.max(self.dist_matrix)
        self.S = max_dist - self.dist_matrix
        np.fill_diagonal(self.S, -self.INF) 
        
        # Messages initialization
        self.tilde_rho = np.zeros((self.N, self.N))
        self.tilde_eta = np.zeros((self.N, self.N))
        self.tilde_phi = np.zeros((self.N, self.N))

    def log(self, msg):
        if self.verbose:
            print(msg)

    def _calc_lambda_sum_bias(self):
        """PDF Eq (1.2) Sum(lambda) 계산 (Beta 역할)"""
        N = self.N
        sum_rho_t = np.sum(self.tilde_rho, axis=1, keepdims=True)
        lambda_sum_bias = -self.tilde_rho + ((N - 1) / N) * sum_rho_t
        return lambda_sum_bias

    def _run_trellis(self):
        N, S, depot = self.N, self.S, self.depot
        bias = self._calc_lambda_sum_bias()
        
        # --- [1] Forward (Alpha) ---
        alpha = [defaultdict(lambda: -self.INF) for _ in range(N + 1)]
        alpha[0][(0, depot)] = 0.0
        
        # 최적화: 루프 내 변수 접근 최소화
        for t in range(N):
            current_bias = bias[t]
            next_alpha = alpha[t+1]
            # items() 복사 오버헤드 방지
            for state, prev_score in alpha[t].items():
                if prev_score < -self.INF / 2: continue
                    
                mask, prev_node = state
                
                # 방문 가능한 노드만 순회 (비트 연산 최적화 가능하지만 가독성 유지)
                # N이 작으므로 range(N) 루프는 빠름
                for i in range(N):
                    if not (mask & (1 << i)):
                        new_mask = mask | (1 << i)
                        val = prev_score + S[prev_node, i] + current_bias[i]
                        
                        if val > next_alpha[(new_mask, i)]:
                            next_alpha[(new_mask, i)] = val

        # --- [2] Backward (Beta) ---
        # Beta는 로직 동일, 코드 생략 없이 최적화만 적용
        beta = [defaultdict(lambda: -self.INF) for _ in range(N + 2)]
        final_mask = self.FULL_MASK
        
        for i in range(N):
            beta[N][(final_mask, i)] = S[i, depot]
            
        for t in range(N - 1, -1, -1):
            curr_bias = bias[t]
            curr_beta = beta[t]
            
            for next_state, next_beta_val in beta[t+1].items():
                if next_beta_val < -self.INF / 2: continue
                next_mask, next_node = next_state
                
                # Xi 값 미리 계산
                xi_val = next_beta_val + curr_bias[next_node]
                
                prev_mask = next_mask & ~(1 << next_node)
                
                # 후보군 추출
                if prev_mask == 0:
                    cands = [depot]
                else:
                    cands = [j for j in range(N) if prev_mask & (1 << j)]
                    
                for prev in cands:
                    val = S[prev, next_node] + xi_val
                    state = (prev_mask, prev)
                    if val > curr_beta[state]:
                        curr_beta[state] = val

        # --- [3] Soft Output (Delta) 최적화 (핵심!) ---
        # 기존: O(N^2 * States) -> 최적화: O(States + N^2)
        tilde_delta = np.zeros((N, N))
        
        for t in range(N):
            # 1. 해당 시간 t의 도시별 최대 점수(City Max Scores)를 한 번의 루프로 수집
            city_max_scores = np.full(N, -self.INF)
            
            # alpha와 beta가 존재하는 상태만 빠르게 스캔
            for state, f_score in alpha[t+1].items():
                if f_score < -self.INF / 2: continue
                if state in beta[t+1]:
                    b_score = beta[t+1][state]
                    if b_score > -self.INF / 2:
                        total = f_score + b_score
                        city = state[1]
                        if total > city_max_scores[city]:
                            city_max_scores[city] = total
            
            # 2. '자기 자신 제외 최대값'을 구하기 위한 전처리
            # 전체 최대값과 두 번째 최대값을 찾음
            sorted_indices = np.argsort(city_max_scores)[::-1] # 내림차순 정렬 인덱스
            best_idx = sorted_indices[0]
            second_best_idx = sorted_indices[1]
            
            global_max = city_max_scores[best_idx]
            global_second = city_max_scores[second_best_idx]

            # 3. Delta 계산 (벡터화)
            # lam_i_for_i = -1/N * rho
            # lam_i_for_j = (N-1)/N * rho
            rho_t = self.tilde_rho[t]
            lam_i_for_i = -(1.0/N) * rho_t
            lam_i_for_j = ((N-1.0)/N) * rho_t
            
            # max_in: 내가 선택된 경우의 최대 점수
            max_in = city_max_scores - lam_i_for_i
            
            # max_out: 내가 선택되지 않았을 때(다른 도시 중) 최대 점수
            # 내가 1등이면 -> 2등 점수 사용, 내가 1등 아니면 -> 1등 점수 사용
            max_out_raw = np.full(N, global_max)
            max_out_raw[best_idx] = global_second # 1등 자리는 2등 점수로 교체
            
            max_out = max_out_raw - lam_i_for_j
            
            # 최종 차이 계산 (Inf 처리 포함)
            diff = np.where(max_in < -self.INF/2, -self.INF,
                            np.where(max_out < -self.INF/2, self.INF, max_in - max_out))
            
            tilde_delta[t] = diff
                
        return alpha, tilde_delta

    def _run_bp(self, tilde_delta):
        """
        Numpy Broadcasting을 이용한 BP 완전 최적화
        O(N^3) -> O(N^2)
        """
        N = self.N
        
        # 1. Omega 계산
        t_omega = self.tilde_phi + tilde_delta
        
        # 2. Eta 계산 (Row 방향 Max Excluding Self)
        # 각 열(Column)에서 특정 행(t)을 제외한 최대값 구하기
        # - 전체 최대값(max1)과 두번째 최대값(max2)을 구해서 처리
        col_max_idx = np.argmax(t_omega, axis=0) # 각 열의 최대값 위치(행 인덱스)
        col_max_val = np.max(t_omega, axis=0)    # 각 열의 최대값
        
        # 두 번째 최대값을 구하기 위해 최대값 위치를 -INF로 잠시 변경
        temp_omega = t_omega.copy()
        for c in range(N):
            temp_omega[col_max_idx[c], c] = -self.INF
        col_second_max = np.max(temp_omega, axis=0)
        
        # new_eta 구성
        # t가 최대값 위치가 아니면 -> 최대값 사용
        # t가 최대값 위치면 -> 두 번째 최대값 사용
        new_eta = np.zeros((N, N))
        for t in range(N):
            # t행이 해당 열(c)의 최대값 위치인지 체크
            is_max_pos = (col_max_idx == t)
            # True면 second_max, False면 max_val
            new_eta[t] = np.where(is_max_pos, col_second_max, col_max_val)
        new_eta = -new_eta # 부호 반전

        # 3. Gamma 계산
        t_gamma = new_eta + tilde_delta
        
        # 4. Phi 계산 (Col 방향 Max Excluding Self)
        # 이번엔 각 행(Row)에서 특정 열(i)을 제외한 최대값
        row_max_idx = np.argmax(t_gamma, axis=1)
        row_max_val = np.max(t_gamma, axis=1)
        
        temp_gamma = t_gamma.copy()
        for r in range(N):
            temp_gamma[r, row_max_idx[r]] = -self.INF
        row_second_max = np.max(temp_gamma, axis=1)
        
        new_phi = np.zeros((N, N))
        for i in range(N):
            is_max_pos = (row_max_idx == i)
            # 전치(Transpose) 주의: new_phi[t, i] 이므로 t 루프 대신 열벡터 연산
            # 여기서는 이중 루프 없이 브로드캐스팅을 위해 전치 사용이 헷갈릴 수 있으므로 
            # 단순하게 열 단위 할당
            new_phi[:, i] = np.where(row_max_idx == i, row_second_max, row_max_val)
            
        new_phi = -new_phi # 부호 반전
        
        # Update with Damping
        self.tilde_eta = self.damping * self.tilde_eta + (1 - self.damping) * new_eta
        self.tilde_phi = self.damping * self.tilde_phi + (1 - self.damping) * new_phi
        self.tilde_rho = self.tilde_eta + self.tilde_phi

    def _extract_path(self, alpha):
        path = []
        curr_mask = self.FULL_MASK
        best_score = -self.INF
        best_last = -1

        # 1) 마지막 도시 선택
        for i in range(self.N):
            state = (curr_mask, i)
            if state in alpha[self.N]:
                score = alpha[self.N][state] + self.S[i, self.depot]
                if score > best_score:
                    best_score = score
                    best_last = i

        if best_last == -1:
            return [], self.INF

        # 초기 상태
        path = [self.depot, best_last]
        curr_node = best_last
        curr_mask = self.FULL_MASK ^ (1 << best_last)

        # 2) 역추적
        for t in range(self.N - 1, 0, -1):

            cands = [j for j in range(self.N) if curr_mask & (1 << j)]  # depot 제거

            best_prev = -1
            best_val = -self.INF

            for prev in cands:
                state = (curr_mask, prev)
                if state in alpha[t]:
                    val = alpha[t][state] + self.S[prev, curr_node]   # bias 제거 (이미 α에 포함됨)

                    if val > best_val:
                        best_val = val
                        best_prev = prev

            if best_prev == -1:
                print(f"Traceback failed at t={t}")
                return [], self.INF

            path.append(best_prev)
            curr_node = best_prev
            curr_mask ^= (1 << best_prev)

        path.append(self.depot)
        path.reverse()

        # cost 계산
        cost = sum(self.dist_matrix[path[k], path[k+1]] for k in range(len(path)-1))
        return path, cost

    def solve(self):
        best_global_path = []
        best_global_cost = self.INF
        
        print(f"Solving TSP (N={self.N}) with Alpha Visualization")
        
        for it in range(self.bp_iterations):
            alpha, tilde_delta = self._run_trellis()
            path, cost = self._extract_path(alpha)
            
            if cost < best_global_cost:
                best_global_cost = cost
                best_global_path = path
                print(f"[Iter {it}] Cost: {cost:.2f} (New Best!) | Path: {path}")
            else:
                if self.verbose:
                     print(f"[Iter {it}] Cost: {cost:.2f} | Path: {path}")

            self._run_bp(tilde_delta)
            
        return best_global_path, best_global_cost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import AffinityPropagation
import concurrent.futures

# -----------------------------------------------------------
# 2. 데이터 로드 및 전처리
# -----------------------------------------------------------
df_dist = pd.read_csv("성북구_휴지통_직선거리행렬.csv", encoding='cp949', index_col=0)
# 거리 행렬이므로 Affinity Propagation을 위해 유사도(음수 거리)로 변환
similarity_matrix = -1 * df_dist.values.astype(float)

# Preference 설정: None이면 median 사용 (클러스터 개수 적절히 조절)
# 노드가 94개이므로 preference를 낮게(음의 절대값을 크게) 잡으면 클러스터가 큼직해짐
# [수정 2] Preference를 상위 90% 값으로 설정 (클러스터 개수 늘리기)
# 값이 클수록(0에 가까울수록) 클러스터가 많이 생깁니다.
percentage = 60
high_preference = np.percentile(similarity_matrix, percentage)
print(f"설정된 Preference 값 (상위 {percentage}%): {high_preference}")
# Clustering 수행
af = AffinityPropagation(affinity='precomputed', preference=high_preference, damping=0.9, random_state=42)
af.fit(similarity_matrix)

labels = af.labels_
cluster_centers_indices = af.cluster_centers_indices_
n_clusters = len(cluster_centers_indices)

print(f"총 {n_clusters}개의 클러스터로 분할되었습니다.")

# -----------------------------------------------------------
# 3. 병렬 TSP 실행 함수 (각 클러스터용)
# -----------------------------------------------------------
def solve_cluster_tsp(cluster_id, members_indices, full_dist_df):
    """
    특정 클러스터에 속한 노드들만의 거리 행렬을 뽑아서 TSP를 풉니다.
    """
    # 1. 서브 거리 행렬 추출 (해당 클러스터 멤버들끼리만)
    # iloc을 사용하여 해당 인덱스들만 뽑아냅니다.
    sub_dist = full_dist_df.iloc[members_indices, members_indices]
    
    # 2. Solver 초기화 및 실행
    solver = TSPHypercubeBCJR_SOVA(sub_dist)
    local_path, cost = solver.run()
    
    # 3. Local Index -> Global Index(실제 장소명) 변환
    # solver는 0, 1, 2... 리턴하므로 이를 실제 멤버의 인덱스로 매핑해야 함
    global_path_indices = [members_indices[i] for i in local_path]
    global_path_names = [sub_dist.index[i] for i in local_path]
    
    return {
        "cluster_id": cluster_id,
        "local_path": local_path,
        "global_path_indices": global_path_indices, # 전체 행렬 기준 인덱스
        "path_names": global_path_names,            # 장소 이름
        "cost": cost,
        "head_node": members_indices[0] # 편의상 첫번째 멤버를 임시 헤드로 간주하거나 AP center 사용
    }

# -----------------------------------------------------------
# 4. 실행 로직 (Local TSP -> Head TSP)
# -----------------------------------------------------------
cluster_results = {}

# (1) 각 클러스터별 TSP 병렬 실행
print("\n--- [Step 1] 각 클러스터 내부 TSP 병렬 수행 중 ---")
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    
    for i in range(n_clusters):
        # 현재 클러스터에 속한 노드들의 전체 데이터프레임상 인덱스 찾기
        # 주의: AP의 cluster_centers_indices_[i]가 그 클러스터의 대표(Head)입니다.
        # Head를 맨 앞에 두면 Solver가 0번을 Depot으로 인식할 때 유리합니다.
        
        center_idx = cluster_centers_indices[i]
        member_indices = np.where(labels == i)[0].tolist()
        
        # Head(Center)를 리스트 맨 앞으로 이동 (Depot 역할 수행을 위해)
        if center_idx in member_indices:
            member_indices.remove(center_idx)
            member_indices.insert(0, center_idx)
            
        futures.append(executor.submit(solve_cluster_tsp, i, member_indices, df_dist))

    for future in concurrent.futures.as_completed(futures):
        res = future.result()
        cluster_results[res['cluster_id']] = res
        print(f"Cluster {res['cluster_id']} 완료 (비용: {res['cost']}m, 노드 수: {len(res['local_path'])-1}개)")

# (2) Cluster Head끼리의 TSP 실행
print("\n--- [Step 2] Cluster Head 간의 TSP 수행 ---")

# AP가 찾아준 중심점(Center)들의 인덱스
head_indices = cluster_centers_indices 
head_sub_dist = df_dist.iloc[head_indices, head_indices]

head_solver = TSPHypercubeBCJR_SOVA(head_sub_dist)
head_path_local, head_cost = head_solver.run()

# Head 경로 매핑
head_path_global_indices = [head_indices[i] for i in head_path_local]
head_path_names = [head_sub_dist.index[i] for i in head_path_local]

print(f"Head TSP 경로 비용: {head_cost}m")
print(f"Head 방문 순서: {' -> '.join(head_path_names)}")

# -----------------------------------------------------------
# 5. 최종 결과 확인
# -----------------------------------------------------------
print("\n[최종 요약]")
total_distance = head_cost + sum([res['cost'] for res in cluster_results.values()])
print(f"전체 예상 이동 거리 합계(단순 합): {total_distance}m")
# 주의: 단순 합은 '이동 -> 클러스터 순회 -> 복귀 -> 다음 이동'을 가정한 것으로,
# 실제로는 Head 경로와 내부 경로를 잇는 'Stitching' 비용 최적화가 추가로 필요할 수 있습니다.

설정된 Preference 값 (상위 60%): -1758.0
총 14개의 클러스터로 분할되었습니다.

--- [Step 1] 각 클러스터 내부 TSP 병렬 수행 중 ---
Cluster 8 완료 (비용: 3707.0m, 노드 수: 5개)
Cluster 6 완료 (비용: 1850.0m, 노드 수: 7개)
Cluster 5 완료 (비용: 1370.0m, 노드 수: 7개)
Cluster 10 완료 (비용: 1972.0m, 노드 수: 4개)
Cluster 9 완료 (비용: 2116.0m, 노드 수: 4개)
Cluster 7 완료 (비용: 1462.0m, 노드 수: 6개)
Cluster 2 완료 (비용: 323.0m, 노드 수: 3개)
Cluster 4 완료 (비용: 2243.0m, 노드 수: 6개)
Cluster 1 완료 (비용: 3096.0m, 노드 수: 7개)
Cluster 11 완료 (비용: 1464.0m, 노드 수: 4개)
Cluster 12 완료 (비용: 3229.0m, 노드 수: 7개)
Cluster 13 완료 (비용: 2172.0m, 노드 수: 8개)
Cluster 3 완료 (비용: 2623.0m, 노드 수: 12개)
Cluster 0 완료 (비용: 2110.0m, 노드 수: 13개)

--- [Step 2] Cluster Head 간의 TSP 수행 ---
Head TSP 경로 비용: 16971.0m
Head 방문 순서: 성신여대입구 버스정류장 -> 성북문화원 -> 청덕초교 -> 정릉역1번출구 -> 프린트카페 길음점 -> 리안헤어 종암하월곡점 -> 이마트24 성북장위점 -> 우리은행 석계역지점 -> 힐링마사지 -> 이마트24 뉴월곡역점 -> 새종암새마을금고 제1지점 -> KB국민은행 종암동종합금융센터 -> 미스터국밥 -> 우리네흑염소삼계탕추어탕 -> 성신여대입구 버스정류장

[최종 요약]
전체 예상 이동 거리 합계(단순 합): 46708.0m


: 

In [ ]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.cluster import AffinityPropagation
from sklearn.metrics.pairwise import haversine_distances
import concurrent.futures

# =========================================================
# 2. 데이터 로드 및 전처리
# =========================================================
print("데이터 로드 및 거리 계산 중...")
try:
    df_coords = pd.read_csv("성북구_가로휴지통_전처리완료.csv", encoding='cp949')
except:
    df_coords = pd.read_csv("성북구_가로휴지통_전처리완료.csv", encoding='utf-8')

# 위도/경도 -> 라디안 변환 -> 거리 행렬(미터) 계산
coords_rad = np.radians(df_coords[['위도', '경도']].values)
dist_matrix = haversine_distances(coords_rad) * 6371000 # 지구 반지름 약 6371km

# 데이터프레임으로 변환 (인덱싱 편의를 위해)
df_dist = pd.DataFrame(dist_matrix, index=df_coords.index, columns=df_coords.index)

# =========================================================
# 3. 클러스터링 (Affinity Propagation)
# =========================================================
print("클러스터링 수행 중...")
similarity_matrix = -dist_matrix # 거리가 짧을수록 유사도 높음
pref = np.percentile(similarity_matrix, 50) # 상위 90% Preference 적용 (잘게 쪼개기)

af = AffinityPropagation(affinity='precomputed', preference=pref, damping=0.9, random_state=42)
af.fit(similarity_matrix)

labels = af.labels_
cluster_centers_indices = af.cluster_centers_indices_
n_clusters = len(cluster_centers_indices)
print(f"-> 총 {n_clusters}개 구역으로 분할됨")

# =========================================================
# 4. 계층적 TSP 계산 (Parallel)
# =========================================================
print("TSP 경로 계산 중...")

def solve_cluster(cluster_id, member_indices):
    # 서브 거리 행렬 추출
    sub_dist = df_dist.iloc[member_indices, member_indices]
    
    # TSP 풀기
    solver = TSPHypercubeBCJR_SOVA(sub_dist)
    local_path, cost = solver.run()
    
    # Local Index -> Global Index 변환
    global_path = [member_indices[i] for i in local_path]
    
    return cluster_id, {
        'global_path_indices': global_path,
        'head_node': member_indices[0], # 0번 인덱스가 Head(Center)임
        'cost': cost
    }

cluster_results = {}

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for i in range(n_clusters):
        center_idx = cluster_centers_indices[i]
        member_indices = np.where(labels == i)[0].tolist()
        
        # Head를 맨 앞으로 (Depot 역할)
        if center_idx in member_indices:
            member_indices.remove(center_idx)
            member_indices.insert(0, center_idx)
            
        futures.append(executor.submit(solve_cluster, i, member_indices))

    for future in concurrent.futures.as_completed(futures):
        cid, res = future.result()
        cluster_results[cid] = res

# [Head TSP] 각 구역 대표끼리의 경로 계산
head_sub_dist = df_dist.iloc[cluster_centers_indices, cluster_centers_indices]
head_solver = TSPHypercubeBCJR_SOVA(head_sub_dist)
head_path_local, head_cost = head_solver.run()

# ★★★ 여기서 NameError 방지를 위해 head_global_path를 명확히 정의 ★★★
head_global_path = [cluster_centers_indices[i] for i in head_path_local]

# =========================================================
# 5. 지도 시각화 (Folium)
# =========================================================
print("지도 생성 중...")

# (1) 색상 매핑 준비 (Folium 이름 <-> HEX 코드)
folium_color_map = {
    'red': '#d63e2a', 'blue': '#38aadd', 'green': '#72b026', 'purple': '#d252b9',
    'orange': '#f69730', 'darkred': '#a23336', 'lightred': '#ff8e7f', 'beige': '#ffcb92',
    'darkblue': '#0067a3', 'darkgreen': '#728224', 'cadetblue': '#436978',
    'darkpurple': '#5b396b', 'pink': '#ff91ea', 'lightblue': '#8adaff',
    'lightgreen': '#bbf970', 'gray': '#575757', 'black': '#303030'
}
color_keys = list(folium_color_map.keys())

# 지도 초기화
center_lat = df_coords['위도'].mean()
center_lon = df_coords['경도'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=14)

# (2) 각 클러스터 그리기
for cluster_id, res in cluster_results.items():
    # 색상 선택
    color_name = color_keys[cluster_id % len(color_keys)]
    hex_color = folium_color_map[color_name]

    # 좌표 추출
    path_coords = []
    for global_idx in res['global_path_indices']:
        lat = df_coords.iloc[global_idx]['위도']
        lon = df_coords.iloc[global_idx]['경도']
        path_coords.append([lat, lon])
    
    # 경로 (실선)
    folium.PolyLine(
        locations=path_coords,
        color=hex_color,
        weight=4,
        opacity=0.8,
        tooltip=f"Cluster {cluster_id}"
    ).add_to(m)

    # 마커
    head_node_idx = res['head_node']
    for global_idx in res['global_path_indices']:
        lat = df_coords.iloc[global_idx]['위도']
        lon = df_coords.iloc[global_idx]['경도']
        place_name = df_coords.iloc[global_idx]['찾은장소명']
        
        if global_idx == head_node_idx:
            # Head (휴지통 아이콘)
            folium.Marker(
                location=[lat, lon],
                popup=f"<b>[Head C{cluster_id}]</b> {place_name}",
                icon=folium.Icon(color=color_name, icon='trash', prefix='fa'),
                zIndexOffset=1000
            ).add_to(m)
        else:
            # 일반 노드 (원형)
            folium.CircleMarker(
                location=[lat, lon],
                radius=6,
                color=hex_color,
                fill=True,
                fill_color='white',
                fill_opacity=1.0,
                popup=place_name
            ).add_to(m)

# (3) 전체 Head 경로 그리기 (검은색 점선)
head_path_coords = []
for global_idx in head_global_path:
    lat = df_coords.iloc[global_idx]['위도']
    lon = df_coords.iloc[global_idx]['경도']
    head_path_coords.append([lat, lon])

folium.PolyLine(
    locations=head_path_coords,
    color='black',
    weight=5,
    opacity=1.0,
    dash_array='10, 15', # 점선 효과
    tooltip="Global Head Path (Dashed)"
).add_to(m)

# 저장
output_file = "성북구_휴지통_최종_점선_완성본.html"
m.save(output_file)
print(f"완료되었습니다! '{output_file}' 파일을 확인해주세요.")

데이터 로드 및 거리 계산 중...
클러스터링 수행 중...
-> 총 10개 구역으로 분할됨
TSP 경로 계산 중...
지도 생성 중...
완료되었습니다! '성북구_휴지통_최종_점선_완성본.html' 파일을 확인해주세요.


: 

In [ ]:
import pandas as pd
import numpy as np
import requests
import folium
import json
import time
from sklearn.cluster import AffinityPropagation
from sklearn.metrics.pairwise import haversine_distances

# =========================================================
# [설정] 카카오 API 키 입력
# =========================================================
KAKAO_API_KEY = "29cf96c3bebe9f8caec569384f45f2b4"

# =========================================================
# 2. 유틸리티 함수 (API & 거리 & 형상 계산)
# =========================================================
def get_road_distance(start_x, start_y, end_x, end_y):
    """단순 거리(미터)만 빠르게 조회"""
    url = "https://apis-navi.kakaomobility.com/v1/directions"
    params = {
        "origin": f"{start_x},{start_y}", "destination": f"{end_x},{end_y}",
        "priority": "RECOMMEND", "summary": True
    }
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        resp = requests.get(url, params=params, headers=headers)
        if resp.status_code == 200:
            routes = resp.json().get('routes')
            if routes: return routes[0]['summary']['distance']
    except Exception as e:
        print(f"Err: {e}")
    return haversine_distance(start_y, start_x, end_y, end_x)

def get_kakao_route_path(start_x, start_y, end_x, end_y):
    """
    [추가됨] 출발지 -> 도착지 경로의 '상세 좌표 리스트(도로 형상)' 반환
    지도에 도로 모양대로 선을 그리기 위해 사용
    """
    url = "https://apis-navi.kakaomobility.com/v1/directions"
    params = {
        "origin": f"{start_x},{start_y}",
        "destination": f"{end_x},{end_y}",
        "priority": "RECOMMEND"
        # summary=True를 빼면 상세 경로(vertexes)가 나옵니다.
    }
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    
    path_coords = []
    try:
        resp = requests.get(url, params=params, headers=headers)
        if resp.status_code == 200:
            routes = resp.json().get('routes')
            if routes:
                sections = routes[0]['sections']
                for section in sections:
                    for road in section['roads']:
                        vertexes = road['vertexes']
                        # vertexes는 [x, y, x, y...] 순서이므로 (lat, lon)으로 변환
                        for k in range(0, len(vertexes), 2):
                            path_coords.append([vertexes[k+1], vertexes[k]]) # lat, lon
        return path_coords
    except Exception as e:
        print(f"Path Err: {e}")
        return [[start_y, start_x], [end_y, end_x]] # 실패시 직선

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371000
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2) * np.sin(dlambda/2)**2
    return 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1-a))

def build_road_dist_matrix(coords_df):
    n = len(coords_df)
    matrix = np.zeros((n, n))
    indices = coords_df.index.tolist()
    print(f"  - 도로 거리 행렬 생성 ({n}x{n})...")
    for i in range(n):
        for j in range(n):
            if i != j:
                src, dst = coords_df.iloc[i], coords_df.iloc[j]
                matrix[i][j] = get_road_distance(src['경도'], src['위도'], dst['경도'], dst['위도'])
    return pd.DataFrame(matrix, index=indices, columns=indices)

# =========================================================
# 3. 메인 로직 실행
# =========================================================
if __name__ == "__main__":
    # [1] 데이터 로드
    print("[1] 데이터 로드...")
    try: df = pd.read_csv("성북구_가로휴지통_전처리완료.csv", encoding='cp949')
    except: df = pd.read_csv("성북구_가로휴지통_전처리완료.csv", encoding='utf-8')

    # [2] 클러스터링
    print("[2] Affinity Propagation 클러스터링...")
    coords_rad = np.radians(df[['위도', '경도']].values)
    sim = -(haversine_distances(coords_rad) * 6371000)
    
    # 클러스터 개수 조절 (너무 많으면 np.min(sim) 사용 권장)
    pref = np.percentile(sim, 50) 
    
    af = AffinityPropagation(affinity='precomputed', preference=pref, damping=0.9, random_state=42).fit(sim)
    labels, centers = af.labels_, af.cluster_centers_indices_
    n_clusters = len(centers)
    print(f"  -> {n_clusters}개 클러스터 생성")

    # [3] 경로 계산
    print("[3] 경로 계산 (API 도로 거리)...")
    cluster_results = {}
    total_dist_sova = 0
    total_dist_nn = 0

    for i in range(n_clusters):
        print(f"  -> Cluster {i} 처리 중...")
        m_idxs = np.where(labels == i)[0].tolist()
        c_idx = centers[i]
        if c_idx in m_idxs: m_idxs.remove(c_idx); m_idxs.insert(0, c_idx)
        
        sub_df = df.iloc[m_idxs]
        road_mat = build_road_dist_matrix(sub_df)
        
        # [A] SOVA
        p_opt, c_opt = TSPSolverSOVA(road_mat).solve()
        gp_opt = [m_idxs[x] for x in p_opt]
        total_dist_sova += c_opt 
        
        # [B] NN
        p_nn, c_nn = TSP_NearestNeighbor(road_mat).run()
        gp_nn = [m_idxs[x] for x in p_nn]
        total_dist_nn += c_nn 
        
        cluster_results[i] = {'head': c_idx, 'opt': (gp_opt, c_opt), 'nn': (gp_nn, c_nn)}

    # [4] Head TSP
    print("[4] Head 경로 계산...")
    head_mat = build_road_dist_matrix(df.iloc[centers])
    
    hp_opt, hc_opt = TSPSolverSOVA(head_mat).solve()
    hgp_opt = [centers[x] for x in hp_opt]
    total_dist_sova += hc_opt 
    
    hp_nn, hc_nn = TSP_NearestNeighbor(head_mat).run()
    hgp_nn = [centers[x] for x in hp_nn]
    total_dist_nn += hc_nn 

    # ★ 거리 출력
    print("\n" + "="*40)
    print(f" [최종 결과 비교]")
    print(f" 1. SOVA (점선) 총 이동 거리: {total_dist_sova:,.0f} m")
    print(f" 2. NN   (실선) 총 이동 거리: {total_dist_nn:,.0f} m")
    print("="*40 + "\n")

    # [5] 시각화 (도로 형상 적용)
    print("[5] 지도 생성 (도로 형상 적용)...")
    folium_colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'darkblue', 'cadetblue', 'black', 'pink']
    folium_hex = {'red':'#d63e2a', 'blue':'#38aadd', 'green':'#72b026', 'purple':'#d252b9', 'orange':'#f69730', 
                  'darkred':'#a23336', 'darkblue':'#0067a3', 'cadetblue':'#436978', 'black':'#303030', 'pink':'#ff91ea'}
    
    m = folium.Map(location=[df['위도'].mean(), df['경도'].mean()], zoom_start=14)
    
    # (1) 클러스터 내부 (PolyLine 유지 - API 호출 절약)
    for i, res in cluster_results.items():
        c_name = folium_colors[i % len(folium_colors)]
        c_hex = folium_hex.get(c_name, '#333333')
        
        # NN (실선)
        nn_coords = [[df.iloc[x]['위도'], df.iloc[x]['경도']] for x in res['nn'][0]]
        folium.PolyLine(nn_coords, color=c_hex, weight=8, opacity=0.4, tooltip=f"C{i} NN").add_to(m)
        
        # SOVA (점선)
        opt_coords = [[df.iloc[x]['위도'], df.iloc[x]['경도']] for x in res['opt'][0]]
        folium.PolyLine(opt_coords, color=c_hex, weight=3, opacity=1.0, dash_array='5,5', tooltip=f"C{i} SOVA").add_to(m)

        # 마커
        for idx in res['opt'][0]:
            lat, lon = df.iloc[idx]['위도'], df.iloc[idx]['경도']
            if idx == res['head']:
                folium.Marker([lat,lon], popup=f"Head C{i}", icon=folium.Icon(color=c_name, icon='trash', prefix='fa')).add_to(m)
            else:
                folium.CircleMarker([lat,lon], radius=5, color=c_hex, fill=True, fill_color='white').add_to(m)

    # (2) Head 경로 그리기 (★도로 형상 적용★)
    print("   -> Head 경로 도로 형상 다운로드 중...")
    
    # NN Head -> 실선 (빨강/검정 등 구분을 위해 'Gray' 사용하거나 Black 유지)
    full_path_nn = []
    for k in range(len(hgp_nn)-1):
        s_idx, e_idx = hgp_nn[k], hgp_nn[k+1]
        src, dst = df.iloc[s_idx], df.iloc[e_idx]
        seg_path = get_kakao_route_path(src['경도'], src['위도'], dst['경도'], dst['위도'])
        if not seg_path: seg_path = [[src['위도'], src['경도']], [dst['위도'], dst['경도']]]
        full_path_nn.extend(seg_path)
        
    folium.PolyLine(full_path_nn, color='black', weight=8, opacity=0.4, tooltip="Head NN (Road)").add_to(m)
    
    # SOVA Head -> 점선
    full_path_opt = []
    for k in range(len(hgp_opt)-1):
        s_idx, e_idx = hgp_opt[k], hgp_opt[k+1]
        src, dst = df.iloc[s_idx], df.iloc[e_idx]
        seg_path = get_kakao_route_path(src['경도'], src['위도'], dst['경도'], dst['위도'])
        if not seg_path: seg_path = [[src['위도'], src['경도']], [dst['위도'], dst['경도']]]
        full_path_opt.extend(seg_path)

    folium.PolyLine(full_path_opt, color='black', weight=3, opacity=1.0, dash_array='5,5', tooltip="Head SOVA (Road)").add_to(m)

    m.save("성북구_휴지통_최종_도로형상적용2.html")
    print("지도 생성 완료!")

[1] 데이터 로드...
[2] Affinity Propagation 클러스터링...
  -> 10개 클러스터 생성
[3] 경로 계산 (API 도로 거리)...
  -> Cluster 0 처리 중...
  - 도로 거리 행렬 생성 (13x13)...
  -> Cluster 1 처리 중...
  - 도로 거리 행렬 생성 (7x7)...
  -> Cluster 2 처리 중...
  - 도로 거리 행렬 생성 (15x15)...
  -> Cluster 3 처리 중...
  - 도로 거리 행렬 생성 (7x7)...
  -> Cluster 4 처리 중...
  - 도로 거리 행렬 생성 (8x8)...
  -> Cluster 5 처리 중...
  - 도로 거리 행렬 생성 (6x6)...
  -> Cluster 6 처리 중...
  - 도로 거리 행렬 생성 (6x6)...
  -> Cluster 7 처리 중...
  - 도로 거리 행렬 생성 (7x7)...
  -> Cluster 8 처리 중...
  - 도로 거리 행렬 생성 (8x8)...
  -> Cluster 9 처리 중...
  - 도로 거리 행렬 생성 (16x16)...
[4] Head 경로 계산...
  - 도로 거리 행렬 생성 (10x10)...
Solving TSP (N=9) with Alpha Visualization

--------------------------------------------------------------------------------
[Internal] Alpha Matrix Summary (Max Score @ Step t, City i)
   Rows: Time Step (1st..Nth visit)
   Cols: City Index (0..N-1)
--------------------------------------------------------------------------------
[[ 4690.  4440.  7286.  4484.  5774.  6151.  53

In [11]:
import pandas as pd
import folium

# 1. 데이터 불러오기 (파일 경로를 실제 파일 위치로 맞춰주세요)
# 한글 인코딩 문제 발생 시 encoding='utf-8' 또는 'cp949'를 시도해보세요.
filename = '서울특별시 성북구_의류수거함 현황_20240307.csv'
df = pd.read_csv(filename, encoding='cp949')

# 2. 지도의 중심 설정 (데이터의 평균 위도, 경도 사용)
center_lat = df['위도'].mean()
center_lon = df['경도'].mean()

# 3. 지도 객체 생성 (기본 OpenStreetMap 타일 사용)
m = folium.Map(location=[center_lat, center_lon], zoom_start=14)

# 4. 각 의류수거함 위치에 마커 추가
for idx, row in df.iterrows():
    # 위도, 경도 정보가 결측치가 아닌 경우에만 마커 생성
    if pd.notnull(row['위도']) and pd.notnull(row['경도']):
        folium.Marker(
            location=[row['위도'], row['경도']],
            popup=folium.Popup(row['도로명주소'], max_width=300), # 팝업에 주소 표시
            tooltip=row['도로명주소'], # 마우스 오버 시 주소 표시
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)

# 5. 지도를 HTML 파일로 저장
output_file = 'clothing_bins_map.html'
m.save(output_file)

print(f"지도가 {output_file} 파일로 저장되었습니다. 웹 브라우저로 열어보세요.")

지도가 clothing_bins_map.html 파일로 저장되었습니다. 웹 브라우저로 열어보세요.


In [13]:
import pandas as pd
import numpy as np
import requests
import folium
import json
import time
from sklearn.cluster import AffinityPropagation
from sklearn.metrics.pairwise import haversine_distances
from concurrent.futures import ThreadPoolExecutor, as_completed
%pip install tqdm
from tqdm import tqdm  # 진행상황 표시용 (없으면 pip install tqdm)

# [주의] 사용자분의 기존 TSP 클래스(TSPHypercubeBCJR_SOVA, TSP_NearestNeighbor 등)가 
# 반드시 이 코드 위쪽에 선언되어 있거나 import 되어 있어야 합니다.

# =========================================================
# [설정] 카카오 API 키 입력
# =========================================================
KAKAO_API_KEY = "YOUR_KAKAO_API_KEY"  # 실제 키로 변경하세요

# =========================================================
# 유틸리티 함수들 (기존과 동일)
# =========================================================
def get_road_distance(start_x, start_y, end_x, end_y):
    # ... (기존 코드 유지) ...
    url = "https://apis-navi.kakaomobility.com/v1/directions"
    params = {
        "origin": f"{start_x},{start_y}", "destination": f"{end_x},{end_y}",
        "priority": "RECOMMEND", "summary": True
    }
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        resp = requests.get(url, params=params, headers=headers)
        if resp.status_code == 200:
            routes = resp.json().get('routes')
            if routes: return routes[0]['summary']['distance']
    except Exception:
        pass
    return haversine_distance(start_y, start_x, end_y, end_x)

def get_kakao_route_path(start_x, start_y, end_x, end_y):
    # ... (기존 코드 유지) ...
    # (내용 생략 - 위와 동일)
    pass 

def haversine_distance(lat1, lon1, lat2, lon2):
    # ... (기존 코드 유지) ...
    R = 6371000
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2) * np.sin(dlambda/2)**2
    return 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1-a))

def build_road_dist_matrix(coords_df):
    # ... (기존 코드 유지) ...
    n = len(coords_df)
    matrix = np.zeros((n, n))
    indices = coords_df.index.tolist()
    # 병렬 처리 중 print가 섞일 수 있으므로 제거하거나 최소화
    for i in range(n):
        for j in range(n):
            if i != j:
                src, dst = coords_df.iloc[i], coords_df.iloc[j]
                matrix[i][j] = get_road_distance(src['경도'], src['위도'], dst['경도'], dst['위도'])
    return pd.DataFrame(matrix, index=indices, columns=indices)

# =========================================================
# [핵심] 병렬 처리를 위한 작업 함수 정의
# =========================================================
def process_cluster(cluster_id, labels, centers, df):
    """
    하나의 클러스터에 대해 거리 행렬을 만들고 TSP를 수행하는 함수
    """
    try:
        # 해당 클러스터에 속한 인덱스 추출
        m_idxs = np.where(labels == cluster_id)[0].tolist()
        c_idx = centers[cluster_id]
        
        # 중심점을 리스트의 맨 앞으로 이동 (Head)
        if c_idx in m_idxs: 
            m_idxs.remove(c_idx)
        m_idxs.insert(0, c_idx)
        
        # 서브 데이터프레임 생성
        sub_df = df.iloc[m_idxs]
        
        # [API 호출 구간] 거리 행렬 생성
        road_mat = build_road_dist_matrix(sub_df)
        
        # [TSP 알고리즘 수행]
        # 주의: 사용자 정의 클래스(TSPHypercubeBCJR_SOVA, TSP_NearestNeighbor)가 정의되어 있어야 함
        
        # [A] SOVA
        # (TSP 클래스 정의가 코드에 포함되어 있다고 가정)
        p_opt, c_opt = TSPHypercubeBCJR_SOVA(road_mat).run()
        gp_opt = [m_idxs[x] for x in p_opt] # 전체 인덱스로 변환
        
        # [B] NN
        p_nn, c_nn = TSP_NearestNeighbor(road_mat).run()
        gp_nn = [m_idxs[x] for x in p_nn]   # 전체 인덱스로 변환
        
        return {
            'cluster_id': cluster_id,
            'head': c_idx,
            'opt': (gp_opt, c_opt),
            'nn': (gp_nn, c_nn),
            'status': 'success'
        }
    except Exception as e:
        print(f"Error in Cluster {cluster_id}: {e}")
        return {'cluster_id': cluster_id, 'status': 'fail'}

# =========================================================
# 3. 메인 로직 실행
# =========================================================
if __name__ == "__main__":
    # [1] 데이터 로드
    print("[1] 데이터 로드...")
    try: df = pd.read_csv("서울특별시 성북구_의류수거함 현황_20240307.csv", encoding='cp949')
    except: df = pd.read_csv("서울특별시 성북구_의류수거함 현황_20240307.csv", encoding='utf-8')

    # [2] 클러스터링
    print("[2] Affinity Propagation 클러스터링...")
    coords_rad = np.radians(df[['위도', '경도']].values)
    sim = -(haversine_distances(coords_rad) * 6371000)
    pref = np.percentile(sim, 50) 
    
    af = AffinityPropagation(affinity='precomputed', preference=pref, damping=0.9, random_state=42).fit(sim)
    labels, centers = af.labels_, af.cluster_centers_indices_
    n_clusters = len(centers)
    print(f"  -> {n_clusters}개 클러스터 생성")

    # [3] 경로 계산 (병렬 처리 적용)
    print(f"[3] 경로 계산 시작 (병렬 처리: {n_clusters}개 클러스터)...")
    
    cluster_results = {}
    total_dist_sova = 0
    total_dist_nn = 0

    # max_workers: 동시에 실행할 스레드 수 (API 제한 고려하여 4~8 정도 추천)
    # 카카오 API 무료 사용량 제한이 있으니 너무 높게 설정하면 에러 날 수 있음
    MAX_WORKERS = 8 

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # 각 클러스터 작업을 스레드풀에 등록
        futures = [
            executor.submit(process_cluster, i, labels, centers, df) 
            for i in range(n_clusters)
        ]
        
        # 작업 완료되는 대로 결과 수집 (tqdm으로 진행바 표시)
        for future in tqdm(as_completed(futures), total=n_clusters, desc="Calculating Routes"):
            res = future.result()
            if res['status'] == 'success':
                cid = res['cluster_id']
                cluster_results[cid] = {
                    'head': res['head'], 
                    'opt': res['opt'], 
                    'nn': res['nn']
                }
                # 총 거리 합산
                total_dist_sova += res['opt'][1]
                total_dist_nn += res['nn'][1]

    # [4] Head TSP (이 부분은 데이터가 작으므로 그냥 단일 실행)
    print("[4] Head 경로 계산...")
    head_mat = build_road_dist_matrix(df.iloc[centers])
    
    # (주의: Head TSP용 클래스 이름이 TSPSolverSOVA 인지 TSPHypercubeBCJR_SOVA 인지 확인 필요)
    hp_opt, hc_opt = TSPHypercubeBCJR_SOVA(head_mat).run() # 혹은 .solve()
    hgp_opt = [centers[x] for x in hp_opt]
    total_dist_sova += hc_opt 
    
    hp_nn, hc_nn = TSP_NearestNeighbor(head_mat).run()
    hgp_nn = [centers[x] for x in hp_nn]
    total_dist_nn += hc_nn 

    # ★ 거리 출력
    print("\n" + "="*40)
    print(f" [최종 결과 비교]")
    print(f" 1. SOVA (점선) 총 이동 거리: {total_dist_sova:,.0f} m")
    print(f" 2. NN   (실선) 총 이동 거리: {total_dist_nn:,.0f} m")
    print("="*40 + "\n")
    # [5] 시각화 (도로 형상 적용)
    print("[5] 지도 생성 (도로 형상 적용)...")
    folium_colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'darkblue', 'cadetblue', 'black', 'pink']
    folium_hex = {'red':'#d63e2a', 'blue':'#38aadd', 'green':'#72b026', 'purple':'#d252b9', 'orange':'#f69730', 
                  'darkred':'#a23336', 'darkblue':'#0067a3', 'cadetblue':'#436978', 'black':'#303030', 'pink':'#ff91ea'}
    
    m = folium.Map(location=[df['위도'].mean(), df['경도'].mean()], zoom_start=14)
    
    # (1) 클러스터 내부 (PolyLine 유지 - API 호출 절약)
    for i, res in cluster_results.items():
        c_name = folium_colors[i % len(folium_colors)]
        c_hex = folium_hex.get(c_name, '#333333')
        
        # NN (실선)
        nn_coords = [[df.iloc[x]['위도'], df.iloc[x]['경도']] for x in res['nn'][0]]
        folium.PolyLine(nn_coords, color=c_hex, weight=8, opacity=0.4, tooltip=f"C{i} NN").add_to(m)
        
        # SOVA (점선)
        opt_coords = [[df.iloc[x]['위도'], df.iloc[x]['경도']] for x in res['opt'][0]]
        folium.PolyLine(opt_coords, color=c_hex, weight=3, opacity=1.0, dash_array='5,5', tooltip=f"C{i} SOVA").add_to(m)

        # 마커
        for idx in res['opt'][0]:
            lat, lon = df.iloc[idx]['위도'], df.iloc[idx]['경도']
            if idx == res['head']:
                folium.Marker([lat,lon], popup=f"Head C{i}", icon=folium.Icon(color=c_name, icon='trash', prefix='fa')).add_to(m)
            else:
                folium.CircleMarker([lat,lon], radius=5, color=c_hex, fill=True, fill_color='white').add_to(m)

    # (2) Head 경로 그리기 (★도로 형상 적용★)
    print("   -> Head 경로 도로 형상 다운로드 중...")
    
    # NN Head -> 실선 (빨강/검정 등 구분을 위해 'Gray' 사용하거나 Black 유지)
    full_path_nn = []
    for k in range(len(hgp_nn)-1):
        s_idx, e_idx = hgp_nn[k], hgp_nn[k+1]
        src, dst = df.iloc[s_idx], df.iloc[e_idx]
        seg_path = get_kakao_route_path(src['경도'], src['위도'], dst['경도'], dst['위도'])
        if not seg_path: seg_path = [[src['위도'], src['경도']], [dst['위도'], dst['경도']]]
        full_path_nn.extend(seg_path)
        
    folium.PolyLine(full_path_nn, color='black', weight=8, opacity=0.4, tooltip="Head NN (Road)").add_to(m)
    
    # SOVA Head -> 점선
    full_path_opt = []
    for k in range(len(hgp_opt)-1):
        s_idx, e_idx = hgp_opt[k], hgp_opt[k+1]
        src, dst = df.iloc[s_idx], df.iloc[e_idx]
        seg_path = get_kakao_route_path(src['경도'], src['위도'], dst['경도'], dst['위도'])
        if not seg_path: seg_path = [[src['위도'], src['경도']], [dst['위도'], dst['경도']]]
        full_path_opt.extend(seg_path)

    folium.PolyLine(full_path_opt, color='black', weight=3, opacity=1.0, dash_array='5,5', tooltip="Head SOVA (Road)").add_to(m)

    m.save("성북구_의류수거함_최종_도로형상적용.html")
    print("지도 생성 완료!")

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.
[1] 데이터 로드...
[2] Affinity Propagation 클러스터링...
  -> 19개 클러스터 생성
[3] 경로 계산 시작 (병렬 처리: 19개 클러스터)...


Calculating Routes: 100%|██████████| 19/19 [52:43<00:00, 166.48s/it]


[4] Head 경로 계산...

 [최종 결과 비교]
 1. SOVA (점선) 총 이동 거리: 63,035 m
 2. NN   (실선) 총 이동 거리: 72,729 m

[5] 지도 생성 (도로 형상 적용)...
   -> Head 경로 도로 형상 다운로드 중...
지도 생성 완료!


In [ ]:
import pandas as pd
import numpy as np
import requests
import folium
import json
import time
from sklearn.cluster import AffinityPropagation
from sklearn.metrics.pairwise import haversine_distances
from concurrent.futures import ThreadPoolExecutor, as_completed
%pip install tqdm
from tqdm import tqdm  # 진행상황 표시용 (없으면 pip install tqdm)

# [주의] 사용자분의 기존 TSP 클래스(TSPHypercubeBCJR_SOVA, TSP_NearestNeighbor 등)가 
# 반드시 이 코드 위쪽에 선언되어 있거나 import 되어 있어야 합니다.

# =========================================================
# [설정] 카카오 API 키 입력
# =========================================================
KAKAO_API_KEY = "YOUR_KAKAO_API_KEY"  # 실제 키로 변경하세요

# =========================================================
# 유틸리티 함수들 (기존과 동일)
# =========================================================
def get_road_distance(start_x, start_y, end_x, end_y):
    # ... (기존 코드 유지) ...
    url = "https://apis-navi.kakaomobility.com/v1/directions"
    params = {
        "origin": f"{start_x},{start_y}", "destination": f"{end_x},{end_y}",
        "priority": "RECOMMEND", "summary": True
    }
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        resp = requests.get(url, params=params, headers=headers)
        if resp.status_code == 200:
            routes = resp.json().get('routes')
            if routes: return routes[0]['summary']['distance']
    except Exception:
        pass
    return haversine_distance(start_y, start_x, end_y, end_x)

def get_kakao_route_path(start_x, start_y, end_x, end_y):
    # ... (기존 코드 유지) ...
    # (내용 생략 - 위와 동일)
    pass 

def haversine_distance(lat1, lon1, lat2, lon2):
    # ... (기존 코드 유지) ...
    R = 6371000
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2) * np.sin(dlambda/2)**2
    return 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1-a))

def build_road_dist_matrix(coords_df):
    # ... (기존 코드 유지) ...
    n = len(coords_df)
    matrix = np.zeros((n, n))
    indices = coords_df.index.tolist()
    # 병렬 처리 중 print가 섞일 수 있으므로 제거하거나 최소화
    for i in range(n):
        for j in range(n):
            if i != j:
                src, dst = coords_df.iloc[i], coords_df.iloc[j]
                matrix[i][j] = get_road_distance(src['경도'], src['위도'], dst['경도'], dst['위도'])
    return pd.DataFrame(matrix, index=indices, columns=indices)

# =========================================================
# [핵심] 병렬 처리를 위한 작업 함수 정의
# =========================================================
def process_cluster(cluster_id, labels, centers, df):
    """
    하나의 클러스터에 대해 거리 행렬을 만들고 TSP를 수행하는 함수
    """
    try:
        # 해당 클러스터에 속한 인덱스 추출
        m_idxs = np.where(labels == cluster_id)[0].tolist()
        c_idx = centers[cluster_id]
        
        # 중심점을 리스트의 맨 앞으로 이동 (Head)
        if c_idx in m_idxs: 
            m_idxs.remove(c_idx)
        m_idxs.insert(0, c_idx)
        
        # 서브 데이터프레임 생성
        sub_df = df.iloc[m_idxs]
        
        # [API 호출 구간] 거리 행렬 생성
        road_mat = build_road_dist_matrix(sub_df)
        
        # [TSP 알고리즘 수행]
        # 주의: 사용자 정의 클래스(TSPHypercubeBCJR_SOVA, TSP_NearestNeighbor)가 정의되어 있어야 함
        
        # [A] SOVA
        # (TSP 클래스 정의가 코드에 포함되어 있다고 가정)
        p_opt, c_opt = TSPSolverSOVA(road_mat).solve()
        gp_opt = [m_idxs[x] for x in p_opt] # 전체 인덱스로 변환
        
        # [B] NN
        p_nn, c_nn = TSP_NearestNeighbor(road_mat).run()
        gp_nn = [m_idxs[x] for x in p_nn]   # 전체 인덱스로 변환
        
        return {
            'cluster_id': cluster_id,
            'head': c_idx,
            'opt': (gp_opt, c_opt),
            'nn': (gp_nn, c_nn),
            'status': 'success'
        }
    except Exception as e:
        print(f"Error in Cluster {cluster_id}: {e}")
        return {'cluster_id': cluster_id, 'status': 'fail'}

# =========================================================
# 3. 메인 로직 실행
# =========================================================
if __name__ == "__main__":
    # [1] 데이터 로드
    print("[1] 데이터 로드...")
    try: df = pd.read_csv("서울특별시 성북구_의류수거함 현황_20240307.csv", encoding='cp949')
    except: df = pd.read_csv("서울특별시 성북구_의류수거함 현황_20240307.csv", encoding='utf-8')

    # [2] 클러스터링
    print("[2] Affinity Propagation 클러스터링...")
    coords_rad = np.radians(df[['위도', '경도']].values)
    sim = -(haversine_distances(coords_rad) * 6371000)
    pref = np.percentile(sim, 50) 
    
    af = AffinityPropagation(affinity='precomputed', preference=pref, damping=0.9, random_state=42).fit(sim)
    labels, centers = af.labels_, af.cluster_centers_indices_
    n_clusters = len(centers)
    print(f"  -> {n_clusters}개 클러스터 생성")

    # [3] 경로 계산 (병렬 처리 적용)
    print(f"[3] 경로 계산 시작 (병렬 처리: {n_clusters}개 클러스터)...")
    
    cluster_results = {}
    total_dist_sova = 0
    total_dist_nn = 0

    # max_workers: 동시에 실행할 스레드 수 (API 제한 고려하여 4~8 정도 추천)
    # 카카오 API 무료 사용량 제한이 있으니 너무 높게 설정하면 에러 날 수 있음
    MAX_WORKERS = 8 

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # 각 클러스터 작업을 스레드풀에 등록
        futures = [
            executor.submit(process_cluster, i, labels, centers, df) 
            for i in range(n_clusters)
        ]
        
        # 작업 완료되는 대로 결과 수집 (tqdm으로 진행바 표시)
        for future in tqdm(as_completed(futures), total=n_clusters, desc="Calculating Routes"):
            res = future.result()
            if res['status'] == 'success':
                cid = res['cluster_id']
                cluster_results[cid] = {
                    'head': res['head'], 
                    'opt': res['opt'], 
                    'nn': res['nn']
                }
                # 총 거리 합산
                total_dist_sova += res['opt'][1]
                total_dist_nn += res['nn'][1]

    # [4] Head TSP (이 부분은 데이터가 작으므로 그냥 단일 실행)
    print("[4] Head 경로 계산...")
    head_mat = build_road_dist_matrix(df.iloc[centers])
    
    # (주의: Head TSP용 클래스 이름이 TSPSolverSOVA 인지 TSPHypercubeBCJR_SOVA 인지 확인 필요)
    hp_opt, hc_opt = TSPSolverSOVA(head_mat).solve() # 혹은 .solve()
    hgp_opt = [centers[x] for x in hp_opt]
    total_dist_sova += hc_opt 
    
    hp_nn, hc_nn = TSP_NearestNeighbor(head_mat).run()
    hgp_nn = [centers[x] for x in hp_nn]
    total_dist_nn += hc_nn 

    # ★ 거리 출력
    print("\n" + "="*40)
    print(f" [최종 결과 비교]")
    print(f" 1. SOVA (점선) 총 이동 거리: {total_dist_sova:,.0f} m")
    print(f" 2. NN   (실선) 총 이동 거리: {total_dist_nn:,.0f} m")
    print("="*40 + "\n")
    # [5] 시각화 (도로 형상 적용)
    print("[5] 지도 생성 (도로 형상 적용)...")
    folium_colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'darkblue', 'cadetblue', 'black', 'pink']
    folium_hex = {'red':'#d63e2a', 'blue':'#38aadd', 'green':'#72b026', 'purple':'#d252b9', 'orange':'#f69730', 
                  'darkred':'#a23336', 'darkblue':'#0067a3', 'cadetblue':'#436978', 'black':'#303030', 'pink':'#ff91ea'}
    
    m = folium.Map(location=[df['위도'].mean(), df['경도'].mean()], zoom_start=14)
    
    # (1) 클러스터 내부 (PolyLine 유지 - API 호출 절약)
    for i, res in cluster_results.items():
        c_name = folium_colors[i % len(folium_colors)]
        c_hex = folium_hex.get(c_name, '#333333')
        
        # NN (실선)
        nn_coords = [[df.iloc[x]['위도'], df.iloc[x]['경도']] for x in res['nn'][0]]
        folium.PolyLine(nn_coords, color=c_hex, weight=8, opacity=0.4, tooltip=f"C{i} NN").add_to(m)
        
        # SOVA (점선)
        opt_coords = [[df.iloc[x]['위도'], df.iloc[x]['경도']] for x in res['opt'][0]]
        folium.PolyLine(opt_coords, color=c_hex, weight=3, opacity=1.0, dash_array='5,5', tooltip=f"C{i} SOVA").add_to(m)

        # 마커
        for idx in res['opt'][0]:
            lat, lon = df.iloc[idx]['위도'], df.iloc[idx]['경도']
            if idx == res['head']:
                folium.Marker([lat,lon], popup=f"Head C{i}", icon=folium.Icon(color=c_name, icon='trash', prefix='fa')).add_to(m)
            else:
                folium.CircleMarker([lat,lon], radius=5, color=c_hex, fill=True, fill_color='white').add_to(m)

    # (2) Head 경로 그리기 (★도로 형상 적용★)
    print("   -> Head 경로 도로 형상 다운로드 중...")
    
    # NN Head -> 실선 (빨강/검정 등 구분을 위해 'Gray' 사용하거나 Black 유지)
    full_path_nn = []
    for k in range(len(hgp_nn)-1):
        s_idx, e_idx = hgp_nn[k], hgp_nn[k+1]
        src, dst = df.iloc[s_idx], df.iloc[e_idx]
        seg_path = get_kakao_route_path(src['경도'], src['위도'], dst['경도'], dst['위도'])
        if not seg_path: seg_path = [[src['위도'], src['경도']], [dst['위도'], dst['경도']]]
        full_path_nn.extend(seg_path)
        
    folium.PolyLine(full_path_nn, color='black', weight=8, opacity=0.4, tooltip="Head NN (Road)").add_to(m)
    
    # SOVA Head -> 점선
    full_path_opt = []
    for k in range(len(hgp_opt)-1):
        s_idx, e_idx = hgp_opt[k], hgp_opt[k+1]
        src, dst = df.iloc[s_idx], df.iloc[e_idx]
        seg_path = get_kakao_route_path(src['경도'], src['위도'], dst['경도'], dst['위도'])
        if not seg_path: seg_path = [[src['위도'], src['경도']], [dst['위도'], dst['경도']]]
        full_path_opt.extend(seg_path)

    folium.PolyLine(full_path_opt, color='black', weight=3, opacity=1.0, dash_array='5,5', tooltip="Head SOVA (Road)").add_to(m)

    m.save("성북구_의류수거함_최종_도로형상적용2.html")
    print("지도 생성 완료!")

: 